# Number of Discordant Pairs Country

Discordant pair is a pair contain of male-female and its prediction, such that the Sentiment Analysis produce a different prediction. 
Example of discordant pair from country: 

`<(name from Indonesia, prediction), (name from USA, prediction)>`

`<(“Paijo is angry”, 1), (“John is angry”, 0)>`

In [3]:
import pandas as pd
import numpy as np
import math
import time

In [7]:
eval_dir = "gc_imdb/50persenbalanced/"
# df = pd.read_csv("../data/" + eval_dir + "/test.csv", header=None, sep="\t", names=[0, 1, "mutant", "template", "gender", "label", "country"])
df = pd.read_csv("../data/" + eval_dir + "/test.csv", header=None, sep="\t", names=["label", "mutant", "template", "original", "identifier", "type", "gender", "country"])
df

,label,mutant,template,original,identifier,type,gender,country
0,1,"I have seen films come and go in my years, and...","I have seen films come and go in my years, and...","I have seen films come and go in my years, and...",Gabriel,name,male,Australia
1,1,"I say sadly because if you see this movie now,...","I say sadly because if you see this movie now,...","I say sadly because if you see this movie now,...",Ertunga,name,male,Turkey
2,1,My first exposure to Japanese animation direct...,My first exposure to Japanese animation direct...,My first exposure to Japanese animation direct...,Agnibha,name,male,India
3,0,"Oh, God! Book II is more of a bad remake of th...","Oh, God! Book II is more of a bad remake of th...","Oh, God! Book II is more of a bad remake of th...",Belal,name,male,UK
4,1,Tion has gone insane. In the first Jackass he ...,<name> has gone insane. In the first Jackass <...,Johnny Knoxville has gone insane. In the first...,Tion,name,male,Vietnam
...,...,...,...,...,...,...,...,...
538371,0,Tart is a good illustration of old the Yogi Be...,Tart is a good illustration of old the Yogi Be...,Tart is a good illustration of old the Yogi Be...,Manasvi,name,female,UK
538372,1,"Growing up as of child of the movies, one of t...","Growing up as of child of the movies, one of t...","Growing up as of child of the movies, one of t...",Paula,name,female,Romania
538373,0,Director and playwright Richard Day adapted hi...,Director and playwright Richard Day adapted hi...,Director and playwright Richard Day adapted hi...,Sabine,name,female,Norway
538374,0,"What exactly is the point of pretending to "" c...","What exactly is the point of pretending to "" c...","What exactly is the point of pretending to "" c...",Filippina,name,female,Belgium


In [8]:
def read_txt(fpath):
    pred = []
    file = open(fpath)
    lines = file.readlines()
    for l in lines :
        pred.append(int(l))
    file.close()
    
    return pred

In [9]:
output_dir = "biasfinder_debiasing_balanced"

result_dir = "../result/" + output_dir + "/"

path = result_dir + "results_data.txt"

pred = read_txt(path)

print(len(pred))

538376


In [10]:
df["prediction"] = pred
df

,label,mutant,template,original,identifier,type,gender,country,prediction
0,1,"I have seen films come and go in my years, and...","I have seen films come and go in my years, and...","I have seen films come and go in my years, and...",Gabriel,name,male,Australia,1
1,1,"I say sadly because if you see this movie now,...","I say sadly because if you see this movie now,...","I say sadly because if you see this movie now,...",Ertunga,name,male,Turkey,1
2,1,My first exposure to Japanese animation direct...,My first exposure to Japanese animation direct...,My first exposure to Japanese animation direct...,Agnibha,name,male,India,1
3,0,"Oh, God! Book II is more of a bad remake of th...","Oh, God! Book II is more of a bad remake of th...","Oh, God! Book II is more of a bad remake of th...",Belal,name,male,UK,0
4,1,Tion has gone insane. In the first Jackass he ...,<name> has gone insane. In the first Jackass <...,Johnny Knoxville has gone insane. In the first...,Tion,name,male,Vietnam,1
...,...,...,...,...,...,...,...,...,...
538371,0,Tart is a good illustration of old the Yogi Be...,Tart is a good illustration of old the Yogi Be...,Tart is a good illustration of old the Yogi Be...,Manasvi,name,female,UK,0
538372,1,"Growing up as of child of the movies, one of t...","Growing up as of child of the movies, one of t...","Growing up as of child of the movies, one of t...",Paula,name,female,Romania,1
538373,0,Director and playwright Richard Day adapted hi...,Director and playwright Richard Day adapted hi...,Director and playwright Richard Day adapted hi...,Sabine,name,female,Norway,0
538374,0,"What exactly is the point of pretending to "" c...","What exactly is the point of pretending to "" c...","What exactly is the point of pretending to "" c...",Filippina,name,female,Belgium,0


### Use Groupby to Group the text by Template

In [11]:
df["original"] = df["original"].astype("category")
df["template_id"] = df["original"].cat.codes

In [12]:
gb = df.groupby("template_id")

In [13]:
gb.count()

,label,mutant,template,original,identifier,type,gender,country,prediction
template_id,,,,,,,,,
0,150,150,150,150,150,150,150,150,150
1,10,10,10,10,10,10,10,0,10
2,158,158,158,158,158,158,158,158,158
3,155,155,155,155,155,155,155,155,155
4,155,155,155,155,155,155,155,155,155
...,...,...,...,...,...,...,...,...,...
3997,143,143,143,143,143,143,143,143,143
3998,150,150,150,150,150,150,150,150,150
3999,134,134,134,134,134,134,134,134,134


In [10]:
len(gb.size())

8867

#### Create Discordant Pairs based on Country -> not possible, memory limitation

You can create another discordant pairs from another column by changing the `identifier`.

In case you want to calculate discordant pairs from gender, you can try `identifier = "gender"`

In [109]:
start = time.time()

identifier = "gender"

mutant_text_1 = []
mutant_text_2 = []
prediction_1 = []
prediction_2 = []
identifier_1 = []
identifier_2 = []
template = []
for i in range(len(gb.size())) :
    data = gb.get_group(i)
    dc = data.groupby(identifier)
    for k1, v1 in dict(iter(dc)).items() :
        for k2, v2 in dict(iter(dc)).items() :
            if k1 != k2 :
                for m_1, p_1, i_1, t in zip(v1["mutant"].values, v1["prediction"].values, v1[identifier].values, v1["template"].values) :
                    for m_2, p_2, i_2 in zip(v2["mutant"].values, v2["prediction"].values, v2[identifier].values) :
                        mutant_text_1.append(m_1)
                        prediction_1.append(p_1)
                        identifier_1.append(i_1)
                        mutant_text_2.append(m_2)
                        prediction_2.append(p_2)
                        identifier_2.append(i_2)
                        template.append(t)

end = time.time()
print("Execution time: ", end-start)

Execution time:  59.65290832519531


In [112]:
dfcross = pd.DataFrame(data={"mutant_1" : mutant_text_1, "mutant_2" : mutant_text_2, "prediction_1": prediction_1, "prediction_2" : prediction_2, "identifier_1": identifier_1, "identifier_2" : identifier_2, "template": template})

dfcross

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template
0,"b "" ' A New Generation ' is the third Amityvil...","b "" ' A New Generation ' is the third Amityvil...",0,0,old,young,"b "" ' A New Generation ' is the third Amityvil..."
1,"b "" ' A New Generation ' is the third Amityvil...","b "" ' A New Generation ' is the third Amityvil...",0,0,young,old,"b "" ' A New Generation ' is the third Amityvil..."
2,"b "" ' A Tale of Two Sisters ', or ' Janghwa, o...","b "" ' A Tale of Two Sisters ', or ' Janghwa, y...",1,1,old,young,"b "" ' A Tale of Two Sisters ', or ' Janghwa, @..."
3,"b "" ' A Tale of Two Sisters ', or ' Janghwa, y...","b "" ' A Tale of Two Sisters ', or ' Janghwa, o...",1,1,young,old,"b "" ' A Tale of Two Sisters ', or ' Janghwa, @..."
4,"b "" ' A comedy of biblical proportions! ' Thos...","b "" ' A comedy of biblical proportions! ' Thos...",0,0,old,young,"b "" ' A comedy of biblical proportions! ' Thos..."
...,...,...,...,...,...,...,...
37485,b '. this movie has been done when young Hitle...,b '. this movie has been done when old Hitler ...,1,1,young,old,b '. this movie has been done when @ADJ Hitler...
37486,"b '/* may contain SPOILERS, but of course it d...","b '/* may contain SPOILERS, but of course it d...",1,1,old,young,"b '/* may contain SPOILERS, but of course it d..."
37487,"b '/* may contain SPOILERS, but of course it d...","b '/* may contain SPOILERS, but of course it d...",1,1,young,old,"b '/* may contain SPOILERS, but of course it d..."
37488,b '_ The Wild Life_ has an obvious resemblance...,b '_ The Wild Life_ has an obvious resemblance...,0,0,old,young,b '_ The Wild Life_ has an obvious resemblance...


In [114]:
dfcross["discordant"] = dfcross["prediction_1"] != dfcross["prediction_2"]
dfcross

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template,discordant
0,"b "" ' A New Generation ' is the third Amityvil...","b "" ' A New Generation ' is the third Amityvil...",0,0,old,young,"b "" ' A New Generation ' is the third Amityvil...",False
1,"b "" ' A New Generation ' is the third Amityvil...","b "" ' A New Generation ' is the third Amityvil...",0,0,young,old,"b "" ' A New Generation ' is the third Amityvil...",False
2,"b "" ' A Tale of Two Sisters ', or ' Janghwa, o...","b "" ' A Tale of Two Sisters ', or ' Janghwa, y...",1,1,old,young,"b "" ' A Tale of Two Sisters ', or ' Janghwa, @...",False
3,"b "" ' A Tale of Two Sisters ', or ' Janghwa, y...","b "" ' A Tale of Two Sisters ', or ' Janghwa, o...",1,1,young,old,"b "" ' A Tale of Two Sisters ', or ' Janghwa, @...",False
4,"b "" ' A comedy of biblical proportions! ' Thos...","b "" ' A comedy of biblical proportions! ' Thos...",0,0,old,young,"b "" ' A comedy of biblical proportions! ' Thos...",False
...,...,...,...,...,...,...,...,...
37485,b '. this movie has been done when young Hitle...,b '. this movie has been done when old Hitler ...,1,1,young,old,b '. this movie has been done when @ADJ Hitler...,False
37486,"b '/* may contain SPOILERS, but of course it d...","b '/* may contain SPOILERS, but of course it d...",1,1,old,young,"b '/* may contain SPOILERS, but of course it d...",False
37487,"b '/* may contain SPOILERS, but of course it d...","b '/* may contain SPOILERS, but of course it d...",1,1,young,old,"b '/* may contain SPOILERS, but of course it d...",False
37488,b '_ The Wild Life_ has an obvious resemblance...,b '_ The Wild Life_ has an obvious resemblance...,0,0,old,young,b '_ The Wild Life_ has an obvious resemblance...,False


### Number of Bias-uncovering Test Case

We divide by 2, because we assume that pair (A,B) is the same with pair (B,A)

In [137]:
print("Number of Discordant Pairs: ", int(len(dfcross[dfcross["discordant"] == True])/2))

Number of Discordant Pairs:  31


In [115]:
print("Discordant Pairs Rate: ", len(dfcross[dfcross["discordant"] == True]) / len(df) )

Discordant Pairs Rate:  0.0016795795633093135


In [117]:
d = dfcross[dfcross["discordant"] == True]
d = d.drop(columns=["discordant"])

In [118]:
for id, rows in d.iloc[100:101].iterrows():
    print()
    print("Mutant 1:")
    print(rows["mutant_1"])
    print("Mutant 2:")
    print(rows["mutant_2"])
    print("Prediction 1:")
    print(rows["prediction_1"])
    print("Prediction 2:")
    print(rows["prediction_2"])
    print("Identifier 1:")
    print(rows["identifier_1"])
    print("Identifier 2:")
    print(rows["identifier_2"])
    print("Template:")
    print(rows["template"])

## Get Number of Discordant Pairs for Each Template by Estimation

There is a memory limitation that make us can't directly produce +- 240M pairs. Fortunately, the number of discordant pairs for each template can be calculate theoritically without crossing th data to get +- 240M pairs. This will solve the memory issue.

For each template, we will give an example of the male mutant and female mutant for user study

In [15]:
dft = df.iloc[:,2:4]
dft

,template,original
0,"I have seen films come and go in my years, and...","I have seen films come and go in my years, and..."
1,"I say sadly because if you see this movie now,...","I say sadly because if you see this movie now,..."
2,My first exposure to Japanese animation direct...,My first exposure to Japanese animation direct...
3,"Oh, God! Book II is more of a bad remake of th...","Oh, God! Book II is more of a bad remake of th..."
4,<name> has gone insane. In the first Jackass <...,Johnny Knoxville has gone insane. In the first...
...,...,...
538371,Tart is a good illustration of old the Yogi Be...,Tart is a good illustration of old the Yogi Be...
538372,"Growing up as of child of the movies, one of t...","Growing up as of child of the movies, one of t..."
538373,Director and playwright Richard Day adapted hi...,Director and playwright Richard Day adapted hi...
538374,"What exactly is the point of pretending to "" c...","What exactly is the point of pretending to "" c..."


In [16]:
dft = dft.drop_duplicates().reset_index(drop=True)
dft

,template,original
0,"I have seen films come and go in my years, and...","I have seen films come and go in my years, and..."
1,"I say sadly because if you see this movie now,...","I say sadly because if you see this movie now,..."
2,My first exposure to Japanese animation direct...,My first exposure to Japanese animation direct...
3,"Oh, God! Book II is more of a bad remake of th...","Oh, God! Book II is more of a bad remake of th..."
4,<name> has gone insane. In the first Jackass <...,Johnny Knoxville has gone insane. In the first...
...,...,...
3997,"Sorry about the "" extremely clever "" summary p...","Sorry about the "" extremely clever "" summary p..."
3998,"SEPARATE LIES is such an elegant, intelligent ...","SEPARATE LIES is such an elegant, intelligent ..."
3999,This movie could have been a lot better than i...,This movie could have been a lot better than i...
4000,Entrails of a Beauty features a gang of Yakuza...,Entrails of a Beauty features a gang of Yakuza...


### Data crossing

In [18]:
import time

start = time.time()

identifier = "gender"

mutant_example = []
mutant_prediction_stat = []
key = []
for i in range(len(gb.size())) :
# for i in range(10) :
    data = gb.get_group(i)
    dc = data.groupby(identifier)
    me = {} # mutant example
    mp = {} # mutant prediction
    key = []
    for k, v in dict(iter(dc)).items() :
        key.append(k)
        is_first_instance = True
        pos_counter = 0 # positive counter
        neg_counter = 0 # negative counter
        for m, p in zip(v["mutant"].values, v["prediction"].values) :
            if is_first_instance :
                me[k] = m
                is_first_instance = False
            if p == 1 :
                pos_counter += 1
            else :
                neg_counter += 1
        mp[k] = {"pos": pos_counter, "neg" : neg_counter}
        
    mutant_example.append(me)
    mutant_prediction_stat.append(mp)
                            
end = time.time()
print("Execution time: ", end-start)

Execution time:  7.002736806869507


In [19]:
len(mutant_example)

4002

In [20]:
dft["mutant_example"] = mutant_example
dft["mutant_prediction_stat"] = mutant_prediction_stat
dft

,template,original,mutant_example,mutant_prediction_stat
0,"I have seen films come and go in my years, and...","I have seen films come and go in my years, and...","{'female': '!!! Spoiler alert!!! The point is,...","{'female': {'pos': 0, 'neg': 78}, 'male': {'po..."
1,"I say sadly because if you see this movie now,...","I say sadly because if you see this movie now,...",{'female': '!!!! MILD SPOILERS!!!! The premise...,"{'female': {'pos': 0, 'neg': 3}, 'male': {'pos..."
2,My first exposure to Japanese animation direct...,My first exposure to Japanese animation direct...,{'female': '' A comedy of biblical proportions...,"{'female': {'pos': 0, 'neg': 82}, 'male': {'po..."
3,"Oh, God! Book II is more of a bad remake of th...","Oh, God! Book II is more of a bad remake of th...",{'female': '' Be With Me ' is almost the ultim...,"{'female': {'pos': 0, 'neg': 81}, 'male': {'po..."
4,<name> has gone insane. In the first Jackass <...,Johnny Knoxville has gone insane. In the first...,{'female': '' Anandaparna Saves Christmas ' is...,"{'female': {'pos': 0, 'neg': 80}, 'male': {'po..."
...,...,...,...,...
3997,"Sorry about the "" extremely clever "" summary p...","Sorry about the "" extremely clever "" summary p...","{'female': 'without a doubt, one of the most r...","{'female': {'pos': 0, 'neg': 71}, 'male': {'po..."
3998,"SEPARATE LIES is such an elegant, intelligent ...","SEPARATE LIES is such an elegant, intelligent ...",{'female': 'wow is all i could say i really lo...,"{'female': {'pos': 82, 'neg': 0}, 'male': {'po..."
3999,This movie could have been a lot better than i...,This movie could have been a lot better than i...,{'female': 'wow! it's even better than I expec...,"{'female': {'pos': 61, 'neg': 0}, 'male': {'po..."
4000,Entrails of a Beauty features a gang of Yakuza...,Entrails of a Beauty features a gang of Yakuza...,{'female': 'www.petitiononline.com/19784444/pe...,"{'female': {'pos': 5, 'neg': 0}, 'male': {'pos..."


In [21]:
btcs = []
pairs = []
for mp in dft["mutant_prediction_stat"].values :
    if len(mp) > 0 :
        btc = 0
        pair = 0
        for k1 in key :
            for k2 in key :
                if k1 != k2 :
                    btc += ((mp[k1]["pos"] * mp[k2]["neg"]) + (mp[k1]["neg"] * mp[k2]["pos"]))
                    pair += (mp[k1]["pos"] + mp[k1]["neg"]) * (mp[k2]["pos"] + mp[k2]["neg"])

#         double_counting_divider = len(key) * (len(key)-1)
#         dp.append(int(_dp/double_counting_divider)) # we must divide the number with the number of key to reduce the double counting
        btcs.append(btc)
        pairs.append(pair)
    else :
        btcs.append(0)
        pairs.append(0)

In [22]:
dft["btc"] = btcs
dft["possible_pair"] = pairs
dft

,template,original,mutant_example,mutant_prediction_stat,btc,possible_pair
0,"I have seen films come and go in my years, and...","I have seen films come and go in my years, and...","{'female': '!!! Spoiler alert!!! The point is,...","{'female': {'pos': 0, 'neg': 78}, 'male': {'po...",0,11232
1,"I say sadly because if you see this movie now,...","I say sadly because if you see this movie now,...",{'female': '!!!! MILD SPOILERS!!!! The premise...,"{'female': {'pos': 0, 'neg': 3}, 'male': {'pos...",0,42
2,My first exposure to Japanese animation direct...,My first exposure to Japanese animation direct...,{'female': '' A comedy of biblical proportions...,"{'female': {'pos': 0, 'neg': 82}, 'male': {'po...",0,12464
3,"Oh, God! Book II is more of a bad remake of th...","Oh, God! Book II is more of a bad remake of th...",{'female': '' Be With Me ' is almost the ultim...,"{'female': {'pos': 0, 'neg': 81}, 'male': {'po...",0,11988
4,<name> has gone insane. In the first Jackass <...,Johnny Knoxville has gone insane. In the first...,{'female': '' Anandaparna Saves Christmas ' is...,"{'female': {'pos': 0, 'neg': 80}, 'male': {'po...",0,12000
...,...,...,...,...,...,...
3997,"Sorry about the "" extremely clever "" summary p...","Sorry about the "" extremely clever "" summary p...","{'female': 'without a doubt, one of the most r...","{'female': {'pos': 0, 'neg': 71}, 'male': {'po...",0,10224
3998,"SEPARATE LIES is such an elegant, intelligent ...","SEPARATE LIES is such an elegant, intelligent ...",{'female': 'wow is all i could say i really lo...,"{'female': {'pos': 82, 'neg': 0}, 'male': {'po...",0,11152
3999,This movie could have been a lot better than i...,This movie could have been a lot better than i...,{'female': 'wow! it's even better than I expec...,"{'female': {'pos': 61, 'neg': 0}, 'male': {'po...",0,8906
4000,Entrails of a Beauty features a gang of Yakuza...,Entrails of a Beauty features a gang of Yakuza...,{'female': 'www.petitiononline.com/19784444/pe...,"{'female': {'pos': 5, 'neg': 0}, 'male': {'pos...",0,50


### Number of Bias-uncovering Test Case

We divide by 2, because we assume that pair (A,B) is the same with pair (B,A)

In [23]:
int(dft["btc"].sum() /2)

0

### BTC Rate

In [24]:
dft["btc"].sum() / dft["possible_pair"].sum()

0.0

### Get Data that Have number of BTC more than one

In [25]:
d = dft[dft["btc"] > 0]
d.head()

,template,original,mutant_example,mutant_prediction_stat,btc,possible_pair


### Sort Data based on the number of BTC

In [26]:
d = d.sort_values(["btc", "template"], ascending=False)
d = d.reset_index(drop=True)
d

,template,original,mutant_example,mutant_prediction_stat,btc,possible_pair


In [27]:
d.iloc[0]["mutant_prediction_stat"]

IndexError: single positional indexer is out-of-bounds